In [1]:
from google.cloud import bigquery
client = bigquery.Client('ltvsubscribers')
till_date = '2021-04-01'
sql="""
with Annual_pla_subs as  
(
select A1.*,
--coalesce(Num_Paid_transaction1,0) as Num_Paid_transaction_sprt,
--coalesce(Num_Paid_Refund_transaction,0) as Num_Paid_Refund_transaction,
coalesce(Num_Paid_transaction1,0) as Num_Paid_transaction,
coalesce(Amount_paid,0) as Amount_paid

from `i-dss-cdm-data-dev.ent_vw.subscription_fct` A1
left join (

select 
subscription_guid, 
--count(distinct case when lower( invoice_type_desc ) like '%renewal%' and invoice_status_desc<>'failed' then invoice_guid else null end) as Num_Paid_transaction,
count(distinct case when lower(invoice_status_desc)  in ('paid','closed')  then invoice_guid else null end) as Num_Paid_transaction1,
--count(distinct case when lower(invoice_type_desc) like '%refund%' then invoice_guid else null end) as Num_Paid_Refund_transaction,
sum(case when invoice_status_desc<>'failed' then after_discount_amt else 0 end ) as Amount_paid
from `i-dss-cdm-data-dev.ent_vw.subscription_invoice_fct`
where src_system_id=115
group by subscription_guid
) A2
on A1.subscription_guid=A2.subscription_guid
--left join `ent_summary.att_20160201_present` A3
--on A1.subscription_guid=A3.subscription_guid
where A1.src_system_id=115
and A1.activation_dt  <= '2021-3-31' -- activation date to include the paid subs
and lower(A1.signup_plan_cd) like '%annual%'
and A1.subscription_platform_cd in  ('RECURLY','PSP')
--and A1.signup_coupon_cd not in ('paramountplus','year')
)


select a1.*,
coalesce(Paid_subs_1Invoice,0) as Paid_subs_1Invoice,
coalesce(Paid_subs_2Invoice,0) as Paid_subs_2Invoice,
coalesce(Paid_subs_3Invoice,0) as Paid_subs_3Invoice,
coalesce(Paid_subs_4Invoice,0) as Paid_subs_4Invoice
from
(select 
subscription_platform_cd,
date_trunc(activation_dt, Month) as signup_month,
signup_plan_cd,
count(distinct subscription_guid) as ttl_trial_subs
from Annual_pla_subs
group by 1,2,3) a1

left join (
select 
subscription_platform_cd,
date_trunc(activation_dt, Month) as signup_month,
signup_plan_cd,
count(distinct subscription_guid) as Paid_subs_1Invoice
from Annual_pla_subs
where Amount_paid>40 ------- This was additional filtering using the price point- making sure only Annual Plan starts are considered; Might have to make adjustments for people who came with 50% discount or more as they would have lower price point that this
and Num_Paid_transaction>=1
group by 1,2,3) a2
on a1.signup_month=a2.signup_month
and a1.signup_plan_cd=a2.signup_plan_cd
and a1.subscription_platform_cd= a2.subscription_platform_cd

left join (
select 
subscription_platform_cd,
date_trunc(activation_dt, Month) as signup_month,
signup_plan_cd,
SUM(case when Num_Paid_transaction=2 then Amount_paid else 0 end) Amount_paid_2,
count(distinct subscription_guid) as Paid_subs_2Invoice
from Annual_pla_subs
where Amount_paid>40 
and Num_Paid_transaction>=2
group by 1,2,3) a3
on a1.signup_month=a3.signup_month
and a1.signup_plan_cd=a3.signup_plan_cd
and a1.subscription_platform_cd= a3.subscription_platform_cd

left join (
select 
subscription_platform_cd,
date_trunc(activation_dt, Month) as signup_month,
signup_plan_cd,
count(distinct subscription_guid) as Paid_subs_3Invoice
from Annual_pla_subs
where Amount_paid>40 
and Num_Paid_transaction>=3
group by 1,2,3) a4
on a1.signup_month=a4.signup_month
and a1.signup_plan_cd=a4.signup_plan_cd
and a1.subscription_platform_cd=a4.subscription_platform_cd

left join (
select 
subscription_platform_cd,
date_trunc(activation_dt, Month) as signup_month,
signup_plan_cd,
count(distinct subscription_guid) as Paid_subs_4Invoice
from Annual_pla_subs
where Amount_paid>40 
and Num_Paid_transaction>=4
group by 1,2,3)
a5
on a1.signup_month=a5.signup_month
and a1.signup_plan_cd=a5.signup_plan_cd
and a1.subscription_platform_cd = a5.subscription_platform_cd
order by 1,3,2
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("till_date", "DATE", till_date)
    ]
)
df = client.query(sql, job_config=job_config).to_dataframe()

/Users/agupt0607/miniconda3/envs/env/lib/python3.5/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [15]:
df.to_csv('annual_ltv.csv')

In [7]:
import numpy as np
import pandas as pd
till_date = '2021-04-01'
def annual_CF(df):
    filter_df=df[df['signup_plan_cd'].str.endswith('allaccess_ad_free_annual')]
    cols = filter_df.columns.tolist()
    total_CF= filter_df[cols[3:]].apply(np.sum)
    total_CF['signup_plan_cd'] = 'OVERALL-CF'
    LTV_CF=total_CF['Paid_subs_1Invoice']/total_CF['ttl_trial_subs']*99.99*0.74
    return LTV_CF

def annual_LCS(df):
    filter_df=df[df['signup_plan_cd'].str.endswith('allaccess_annual')]
    cols = filter_df.columns.tolist()
    total_LCS= filter_df[cols[3:]].apply(np.sum)
    total_LCS['signup_plan_cd'] = 'OVERALL-LCS'
    LTV_LCS=total_LCS['Paid_subs_1Invoice']/total_LCS['ttl_trial_subs']*59.99*0.74
    return LTV_LCS

def annual_LTV(df):
    LTV_CF=round(annual_CF(df),2)
    LTV_LCS=round(annual_LCS(df),2)
    overall={'billing_partner':['RECURLY'],'cf_plan':[LTV_CF],'lcs_plan':[LTV_LCS]}
    output=pd.DataFrame(overall).reset_index(drop=True)
    return output

def add_quarter_date(temp,till_date):
    temp['day_dt'] = till_date
    cols = temp.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    output = temp[cols]
    return output

def add_current_date(temp):
    from datetime import date
    today = date.today()
    temp['dw_create_dt'] = pd.to_datetime(date.today())
    return temp

In [8]:
otpt=add_current_date(add_quarter_date(annual_LTV(df),till_date))
otpt

,day_dt,billing_partner,cf_plan,lcs_plan,dw_create_dt
0,2021-04-01,RECURLY,8.76,6.33,2021-06-29


In [9]:
otpt.dtypes

day_dt                     object
billing_partner            object
cf_plan                   float64
lcs_plan                  float64
dw_create_dt       datetime64[ns]
dtype: object

In [14]:
otpt.to_gbq(destination_table='anit_sandbox.pt_ltv_annual_plan_by_quarter',
            project_id='ltvsubscribers',
            if_exists='replace',
           table_schema=[{'name': 'day_dt', 'type': 'DATE'}])
#otpt['day_dt']=otpt['day_dt'].dt.date

In [ ]:
client = bigquery.Client('ltvsubscribers')
table_id = "ltvsubscribers.anit_sandbox.pt_ltv_annual_plan_by_quarter_new"
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("day_dt", bigquery.enums.SqlTypeNames.DATE),
        bigquery.SchemaField("billing_partner", bigquery.enums.SqlTypeNames.STRING),
        # Indexes are written if included in the schema by name.
        
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    # write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    otpt, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)


In [ ]:
bigquery_client = bigquery.Client('ltvsubscribers')

dataset_id = 'anit_sandbox'
dataset_ref = bigquery_client.dataset(dataset_id)
table_ref = dataset_ref.table('pt_ltv_annual_plan_by_quarter_new')

otpt.to_csv('test.csv',index=False)
# You need to cleanup the CSV,to respect the table schema

with open('test.csv', 'rb') as source_file:
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = 'text/csv'
    job = bigquery_client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()    
print('Loaded {} rows into {}:{}.'.format(job.output_rows, dataset_id, 'test_datetime'))
# ^The timestamps inserted from the *.csv by the client library keep their millisecond information^

In [ ]:
pd.read_csv('test.csv')

,day_dt,billing_partner,cf_plan,lcs_plan,dw_create_dt
0,2021-04-01,RECURLY,8.76,6.33,2021-06-29
